In [1]:
# Inital state probability matrix, given
PI = [0.5,0.5]
# PI = {'Healthy': 0.5, 'Depressed': 0.5}
# state transition probability matrix, given
A = [[0.8,0.2],[0.001,0.999]]
# A = {
#     'Healthy' : {'Healthy': 0.8, 'Depressed': 0.2},
#     'Depressed' : {'Healthy': 0.001, 'Depressed': 0.999},
# }
# Emission or observation probability matrix, given
B = [[0.5,0.1,0.3,0.1,0.0],[0.05,0.35,0.2,0.2,0.2]]
# B = {
#     'Healthy' : {'Movement': 0.5, 'Passive': 0.1, 'Active' : 0.3, 'Texting' : 0.1, 'Access' : 0.0},
#     'Depressed' : {'Movement': 0.05, 'Passive': 0.35, 'Active' : 0.2, 'Texting' : 0.2, 'Access' : 0.2},
# }

# state codes of the observed states, dummy
# Observables = [0,1,2,3,4]
# Observables = ["Movement","Passive","Active","Texting","Access"]
#Observables = ["Movement", "Texting", "Movement", "Texting"]
#states = ["Healthy","Depressed"]
states = [0, 1]
Observables = [0,3,0,3]
#Observables = [0,0,2]
#Observables = {"Movement":0, "Passive Social":1, "Active Social":2, "Texting":3, "Access Psych Site":4}

In [2]:
PI_2 = [0.5,0.5]
A_2 = [[0.7,0.3],[0.3,0.7]]
B_2 = [[0.9,0.1],[0.2,0.8]]
states = [0,1]
Observables_2 = [0,0]

# Filtering

In [3]:
import numpy as np 
from numpy import matrix

def forward(PI, A, B, Observables):
    alpha_matrix = []
    curr = {}
    #nex = [0] * len(A)
    for i in range (0, len(A)):
        curr[i] = PI[i] * B[i][0]
    #alpha_matrix.append(curr)
    alpha_matrix.append(list(curr.values()))

    for t in range (1, len(Observables)):
        #nex = [0] * len(A)
        nex = {}
        for i in range (len(A)):
            t_value = Observables[t]
            nex[i] =  sum(curr[j] * A[j][i] * B[i][t_value] for j in range(len(A)))
                #print(nex)
            #nex[i] = nex[i] * B[i][t_value] 
        alpha_matrix.append(list(nex.values()))
        #alpha.insert(t,list(nex.values()))

        curr = nex.copy()
            
    alpha_matrix = np.transpose(alpha_matrix)

    return alpha_matrix

In [4]:
def forward_normalized(PI, A, B, Observables):
    alpha_matrix = []
    curr = {}
    c = {}
    for i in range (0, len(A)):
        curr[i] = PI[i] * B[i][0]
        
    #alpha_matrix.insert(list(curr.values()))
    c[0] = sum(curr[i] for i in range(len(curr)))

    
    c[0] = 1/c[0]
    for i in range (0, len(A)):
        curr[i] = c[0] * curr[i]
    
    alpha_matrix.append(list(curr.values()))

    
    for t in range (1, len(Observables)):
        nex = {}
        nex_norm = {}
        for i in range (len(A)):
                #nex = alpha_matrix[t-1][j]
            t_value = Observables[t]
            nex[i] =  sum(curr[j] * A[j][i] * B[i][t_value] for j in range(len(A)))
        c[t] = 1/sum(nex[k] for k in range(len(nex)))
        #c[t] = 1/c[t]
        for i in range (0,len(A)):
            nex_norm[i] = c[t] * nex[i]
        alpha_matrix.append(list(nex_norm.values()))
        curr = nex_norm
        
      
            
    alpha_matrix = np.transpose(alpha_matrix)
    c_array = list(c.values())

    return alpha_matrix, c_array

# Evaluation

In [5]:
def evaluation_unnormalized(alpha):
    probability = 0
    for i in range(len(alpha)):
        for j in range(0, len(alpha[i])):
             probability += alpha[i][j]        
    return probability

In [6]:
import math
def evaluation_normalized(c):
    probability = -1 * sum(math.log(i) for i in c)
    return probability

# Backward Probability

In [24]:
def backward(PI, A, B, Observables):
    
    beta = []
    curr = {}

    b_prev = {}
    for i, value in enumerate(reversed(Observables)):
        b_curr = {}
        for state in states:
            if i == 0:
                b_curr[state] = 1 
            else:
                b_curr[state] = sum(A[state][l] * B[l][value] * b_prev[l] for l in states)

        beta.insert(0,list(b_curr.values()))
        b_prev = b_curr

    #p_beta = sum(PI[l] * B[l][Observables[0]] * b_curr[l] for l in states)
    
    return np.transpose(beta) #no of initial states * length of observation sequence

In [60]:
def backward_normalized(PI, A, B, Observables):
    alpha,c = forward_normalized(PI, A, B, Observables)
     
    beta = []
    curr = {}

    b_prev = {}
    for i, value in enumerate(reversed(Observables)):
        b_curr = {}
        for state in states:
            if i == 0:
                b_curr[state] = 1 
                b_curr[state] = b_curr[state] * c[i]
            else:
                b_curr[state] = sum(A[state][l] * B[l][value] * b_prev[l] for l in states)

        beta.insert(0,list(b_curr.values()))
        b_prev = b_curr
            
    return np.transpose(beta)

In [40]:
alpha = forward(PI,A,B,Observables)

In [41]:
alpha_norm, c = forward_normalized(PI, A, B, Observables)

In [42]:
c

[3.6363636363636362, 7.857704121722981, 3.9070511989489494, 7.784209968703886]

In [43]:
alpha

array([[0.25      , 0.0200025 , 0.0080085 , 0.00064077],
       [0.025     , 0.014995  , 0.00094903, 0.00050996]])

In [44]:
beta = backward(PI, A, B, Observables)

In [45]:
beta

array([[1.40284091e-02, 3.48180000e-02, 4.10000000e-01, 1.00000000e+00],
       [5.22948455e-04, 1.01209100e-02, 5.04500000e-02, 1.00000000e+00]])

In [61]:
backward_normalized(PI, A, B, Observables)


array([[12.19087967,  3.88701218, 11.71512251,  3.63636364],
       [ 0.45444937,  1.12987824,  1.44153154,  3.63636364]])

# Smoothing by Forward-Backward Algorithm

In [50]:
def smoothed_probability(alpha, beta, noOfStates, noOfTimeSteps):
    matrix = []
    for i in range(len(alpha)):
        curr = {}
        for t in range(len(alpha[i])):
            curr[t] = alpha[i][t] * beta[i][t] / (sum(alpha[j][t] * beta[j][t] for j in range(len(alpha))))
        matrix.append(list(curr.values()) ) 
                
    return matrix

In [51]:
smoothed_probability(alpha,beta,len(alpha), len(beta))

[[0.9962860631362895,
  0.8210784719980339,
  0.9856280045317052,
  0.5568419937407776],
 [0.003713936863710455,
  0.17892152800196623,
  0.014371995468294799,
  0.44315800625922225]]

In [52]:
alpha_2 = forward(PI_2,A_2,B_2,Observables_2)

In [53]:
beta_2 = backward(PI_2, A_2, B_2, Observables_2)

In [54]:
smoothed_probability(alpha_2,beta_2,len(alpha_2), len(beta_2))

[[0.883357041251778, 0.8833570412517782],
 [0.11664295874822192, 0.1166429587482219]]